In [2]:
!pip install python-sat

In [41]:
import os
import math
from copy import deepcopy
import random
import numpy as np
from pysat.solvers import Glucose3
import time

In [42]:
hx = [0, 1, 0, -1, -1, -1, 0, 1, 1]
hy = [0, -1, -1, -1, 0, 1, 1, 1, 0]

## read file

In [43]:
def readFile(filename):
    with open (filename, 'r') as f :
        s = f.readline()
        [m, n] = s.split(" ")
        [m, n] = [int(m), int(n)]

        Matrix =[]
        line = (f.readline()).split()
        while(line):
            Matrix.append([int(i) for i in line])
            line = (f.readline()).split()
    f.close()
    return [Matrix, m, n]
    

In [44]:
def gen_board_num(row,col):
    count=1
    matrix=[]

    for i in range(row):
        tmp=[]
        for j in range(col):
            tmp.append(count)
            count+=1
        matrix.append(tmp)

    return matrix

## Generate & solve CNFs

In [45]:
# Tap hop R1, R2
def check_same_utility(R1,R2):
    if len(R1) != len(R2):
        return False
    R1 = sorted(deepcopy(R1))
    R2 = sorted(deepcopy(R2))
    for i in range(len(R2)):
        if R2[i] != R1[i]:
            return False
    return True

In [46]:
# To hop: C, Tap hop R
def check_same(C,R):
    if len(C) == 0 or len (R) == 0:
        return False
    for i in C:
        if check_same_utility(R,i):
            return True
    return False

In [47]:
def check_same_small(_list, num):
    if len(_list) == 0:
        return False
    for i in _list:
        if i == num:
            return True
    return False

In [48]:
def get_C(adjacentList, k, loop):
    C = []
    for _ in range(loop):
        tmp = []
        for _ in range(k):
            t = random.choice(adjacentList)
            while check_same_small(tmp,t):
                t = random.choice(adjacentList)
                
            tmp.append(t)
        while check_same(C,tmp):
            tmp = []
            for _ in range(k):
                t = random.choice(adjacentList)
                while check_same_small(tmp, t):
                    t = random.choice(adjacentList)
                tmp.append(t)
                
        C.append(tmp)
        
    return C

In [49]:
def get_adjacentList(i,j):
    res=[]

    if i-1 >= 0:
        res.append(board_num[i-1][j])
    if i+1 < len(a):
        res.append(board_num[i+1][j])
    if j-1 >= 0:
        res.append(board_num[i][j-1])
    if j+1 < len(a[0]):
        res.append(board_num[i][j+1])
    if i-1 >= 0 and j-1 >= 0:
        res.append(board_num[i-1][j-1])
    if i-1 >= 0 and j+1 < len(a[0]):
        res.append(board_num[i-1][j+1])
    if i+1 < len(a) and j-1 >= 0:
        res.append(board_num[i+1][j-1])
    if i+1 < len(a) and j+1 < len(a[0]):
        res.append(board_num[i+1][j+1])
    
    return res


In [55]:
def get_clauses(i, j):
    cell=board_num[i][j]
    adjacent_list=get_adjacentList(i,j)
    n=len(adjacent_list)
    k=a[i][j]
    
    clauses=[]
    # blue
    if k > 0 and (k <= n or (k-n) == 1):
        if k == 1:
            for e in adjacent_list:
                clauses.append([-cell,-e])
            tmp=deepcopy(adjacent_list)
            tmp.append(cell)
            clauses.append(tmp)
        else:
            loop_times=int(math.factorial(n)/(math.factorial(k-1)*math.factorial(n-k+1)))
            
            C=get_C(adjacent_list,k-1,loop_times)

            for R in C:
                # forward
                remain=[]
                for e in adjacent_list:
                    if not e in R:
                        remain.append(e)
                tmp=[]
                tmp.append(-cell)
                for m in R:
                    tmp.append(-m)
                for e in remain:
                    tmp_=deepcopy(tmp)
                    tmp_.append(-e)
                    clauses.append(tmp_)
                # reverse
                tmp_=deepcopy(remain)
                tmp_.append(cell)
                clauses.append(tmp_)

                for m in R:
                    tmp_=deepcopy(remain)
                    tmp_.append(m)
                    clauses.append(tmp_)

    #  red
    if k <= n:
        if k == 0:
            clauses.append([-cell])
            for e in adjacent_list:
                clauses.append([-e])
        else:  
            loop_times=int(math.factorial(n)/(math.factorial(k)*math.factorial(n-k)))
            C=get_C(adjacent_list, k,loop_times)
            for R in C:
                # forward
                remain=[]
                for e in adjacent_list:
                    if not e in R:
                        remain.append(e)
                tmp=[]
                for m in R:
                    tmp.append(-m)
                tmp_=deepcopy(tmp)
                tmp_.append(-cell)
                clauses.append(tmp_)

                for e in remain:
                    tmp_=deepcopy(tmp)
                    tmp_.append(-e)
                    clauses.append(tmp_)
                # reverse
                tmp_=deepcopy(remain)
                tmp_.append(cell)
                

                for m in R:
                    tmp__=deepcopy(tmp_)
                    tmp__.append(m)
                    clauses.append(tmp__)

    return clauses

## Use Pysat

In [56]:
import csv

In [57]:
nameinp = "input3.txt"
nameoutp = "output.txt"
f = open(nameoutp, 'w')
[a, m, n] = readFile(nameinp)
color = []
for i in range(m):
    li = []
    for j in range(n):
        li.append(True)
    color.append(li)

print("--------------------------------")
board_num=gen_board_num(m, n)
clauses=[]
for i in range(len(a)):
    for j in range(len(a[0])):
        if a[i][j] > -1:
            clauses+=get_clauses(i,j)
# print("Number of CNFs generated: ", len(clauses))
f.write("Number of CNFs generated: ")
f.write(str(len(clauses)))
f.write('\n')

g = Glucose3()
for it in clauses:
    g.add_clause([int(k) for k in it])


f.write("\nUse pysat:\n")
kt = g.solve()
model = g.get_model()
if kt == True:
    # print(model)
    c = 0
    for i in range(m):
        for j in range(n):
            if model[c] > 0:
                f.write("1 ")
            else:
                f.write("0 ")
            c = c + 1
        f.write('\n')
else:
    f.write("NO SOLUTION\n")
    # print(model)



--------------------------------
